##製品価格を予測します

###週7日目

LoraとQloraの紹介


## Colabを使用するための2つの重要なプロチップのリマインダー：

** Pro-Tip 1：**

すべてのコラブの上部には、いくつかのPIPインストールがあります。これを実行すると、PIPからエラーを受信する場合があります。

> GCSFS 2025.3.2にはFSSPEC == 2025.3.2が必要ですが、FSSPEC 2025.3.0は互換性がありません。

これらのPIP互換性エラーは安全に無視できます。そして、バージョン番号を変更することでそれらを修正しようとするのは魅力的ですが、実際に実際の問題を導入します！

** Pro-Tip 2：**

コラブを実行している最中に、次のようなエラーが発生する可能性があります。

>ランタイムエラー：CUDAは必要ですが、BitsandBytesでは利用できません。 [...]のインストールを検討してください

これは超模倣エラーメッセージです！パッケージのバージョンを変更してみないでください...

これは、GoogleがColabランタイムを切り替えたために実際に起こります。おそらくGoogle Colabが忙しすぎたためです。解決策は次のとおりです。

1。ランタイムメニュー>>ランタイムを切断して削除します
2。新鮮なメニューからコラブをリロードし、メニュー>>すべての出力をクリアします
3.右上のボタンを使用して新しいT4に接続します
4.右上のメニューから「リソースを表示」を選択して、GPUがあることを確認してください
5.コラブ内のセルを上下から、PIPのインストールから始めて再実行します

そして、すべてがうまくいくはずです - そうでなければ、私に尋ねてください！

In [ ]:
# ピップインストール

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0
!pip install -q datasets requests peft

In [ ]:
# 輸入

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [ ]:
# 定数

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"ed-donner/pricer-2024-09-13_13.04.39"

# Qlora微調整のためのハイパーパラメーター

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

### Huggingfaceにログインします

まだHuggingfaceアカウントをお持ちでない場合は、https：//huggingface.coにアクセスしてサインアップしてトークンを作成します。

次に、左のキーアイコンをクリックして、このノートブックの秘密を選択し、トークンとして値を持つ「hf_token」と呼ばれる新しい秘密を追加します。

In [ ]:
# Huggingfaceにログインします

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

##さまざまな量子化を試します


In [ ]:
# 量子化なしでベースモデルをロードします

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## セッションを再開してください！

次のモデルをロードして最後のモデルのキャッシュをクリアするには、ランタイム>>セッションを再起動し、初期セル（インストールとインポート、およびハグFaceログイン）を再度実行する必要があります。

これはGPUをクリーンアウトするためです。

In [ ]:
# 8ビットを使用してベースモデルをロードします

quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## セッションを再開してください！

次のモデルをロードして最後のモデルのキャッシュをクリアするには、ランタイム>>セッションを再起動し、初期セル（インポートとハグFaceログイン）を再度実行する必要があります。

これはGPUをクリーンアウトするためです。

In [ ]:
# 4ビットを使用してトークン剤とベースモデルをロードします

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
base_model

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

In [ ]:
print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
fine_tuned_model

In [ ]:
# 各ターゲットモジュールには、LORA_AとLORA_Bと呼ばれる2つのLORAアダプターマトリックスがあります
# これらは、alpha * lora_a * lora_bを追加することでウェイトを適応できるように設計されています
# 次の寸法を使用して、重みの数を数えましょう。

# 上記のマトリックス寸法を参照してください
lora_q_proj = 4096 * 32 + 4096 * 32
lora_k_proj = 4096 * 32 + 1024 * 32
lora_v_proj = 4096 * 32 + 1024 * 32
lora_o_proj = 4096 * 32 + 4096 * 32

# 各レイヤーが登場します
lora_layer = lora_q_proj + lora_k_proj + lora_v_proj + lora_o_proj

# 32のレイヤーがあります
params = lora_layer * 32

# したがって、MBの合計サイズはです
size = (params * 4) / 1_000_000

print(f"Total number of params: {params:,} and size {size:,.1f}MB")